# 链接数据库

链接数据库需要使用异步的方式,jupyter本身已经异步了,所以直接`await` 调用就好
> 下方`await`红线提示,误报,无视即可

In [1]:
import random

from setting import setting
from app.user.models import User,UserModel
from tortoise import Tortoise

await Tortoise.init(
    config=setting.db_rom_config
)
print('数据库运行成功')

数据库运行成功


## 增

In [58]:
from pydantic import BaseModel

# 创建一条数据,(像数据库表中插入一条数据)
obj = await User.create(account='111', password='123456')
print(obj.account, obj.password)

# 删除数据 
# pk代表表的主键,无论表中主键的字段名是什么,统一用pk表示
await User.filter(pk=obj.pk).delete()

111 $2b$12$vGK6u8aoyyyxT3Pcb1OIG.mc6TxDlIrWho.vBB5leMazMx3c6Lxk6


1

In [59]:
# 通过创建响应模型,可定义数据格式
class UserModelDemo(BaseModel):
    account: str
    password: str
    username: str = None


user = UserModelDemo(account='12312', password='123456')
## user.model_dump() 将响应模型转换为字典格式,通过**解包传递给create参数
obj = await User.create(**user.model_dump())

print(obj.account, obj.password)
await User.filter(pk=obj.pk).delete()

12312 $2b$12$XuDxrQ9iyT1Eu39KB9c5IepsP2SISRC23IulauYIoNZuY3FM25KWC


1

## 查

In [60]:
# all() 查找表中所以数据 
obj = await User.all()
print(obj)

[<User: 1>, <User: 3>, <User: 4>, <User: 6>, <User: 30>]


In [61]:
# .values() 查找所有表中指定数据,需要跟在查询后使用
print(await User.all().values('account'))

[{'account': 'str112323ng'}, {'account': 'string'}, {'account': 'string1'}, {'account': 'arya'}, {'account': '12345'}]


In [62]:
# order_by() 对查询到的结果进行排序  -号表示倒叙
print(await User.all().order_by('-id'))
print(await User.all().order_by('id'))

[<User: 30>, <User: 6>, <User: 4>, <User: 3>, <User: 1>]
[<User: 1>, <User: 3>, <User: 4>, <User: 6>, <User: 30>]


In [63]:
# 分页
# offset() 偏移量  (页码)
# limit()  长度    (每页显示数量)
print(await User.all())
print(await User.all().offset(2).limit(1))

[<User: 1>, <User: 3>, <User: 4>, <User: 6>, <User: 30>]
[<User: 4>]


In [64]:
# 分组
print(await User.all().group_by().values_list('id', 'account'))
print(await User.all().group_by('pk'))

[(1, 'str112323ng'), (3, 'string'), (4, 'string1'), (6, 'arya'), (30, '12345')]
[<User: 1>, <User: 3>, <User: 4>, <User: 6>, <User: 30>]


In [65]:
# count()  统计获得到的数据数量
print(await User.all().count())
print(await User.all().offset(2).limit(10).count())

5
3


### get

In [66]:
# get 根据条件查询符合的对象,条件需唯一,如果查不到,报错提示
print(await User.get(pk=3))
print(await User.get(pk=30))

<User>
<User>


In [67]:
# get_or_none() 根据条件查询符合的对象,条件需唯一,如果查不到,返回None
print(await User.get_or_none(id=3))
print(await User.get_or_none(id=30))

<User>
<User>


In [68]:
# get_or_create() 根据条件查询符合的对象,条件需唯一,如果查不到根据defaults的内容创建对象
print(await User.get_or_create(id=3))
print(await User.get_or_create(id=30,defaults={'account':'12345','password':'12345'}))

(<User: 3>, False)
(<User: 30>, False)


In [18]:
from tortoise.queryset import F
from tortoise.functions import Avg

print(await UserModel.annotate(integral_price=F('integral') * 0.5).values('id','integral'))
# .group_by('gender').values('integral'))

[{'id': 101, 'integral': Decimal('146.00000')}, {'id': 102, 'integral': Decimal('177.00000')}, {'id': 103, 'integral': Decimal('114.00000')}, {'id': 104, 'integral': Decimal('177.00000')}, {'id': 105, 'integral': Decimal('197.00000')}, {'id': 106, 'integral': Decimal('128.00000')}, {'id': 107, 'integral': Decimal('168.00000')}, {'id': 108, 'integral': Decimal('115.00000')}, {'id': 109, 'integral': Decimal('181.00000')}, {'id': 110, 'integral': Decimal('124.00000')}, {'id': 111, 'integral': Decimal('190.00000')}, {'id': 112, 'integral': Decimal('155.00000')}, {'id': 113, 'integral': Decimal('108.00000')}, {'id': 114, 'integral': Decimal('116.00000')}, {'id': 115, 'integral': Decimal('101.00000')}, {'id': 116, 'integral': Decimal('190.00000')}, {'id': 117, 'integral': Decimal('124.00000')}, {'id': 118, 'integral': Decimal('169.00000')}, {'id': 119, 'integral': Decimal('199.00000')}, {'id': 120, 'integral': Decimal('106.00000')}, {'id': 121, 'integral': Decimal('114.00000')}, {'id': 122, 

In [20]:
# first()
# 返回查询到的列表的第一条数据
print(await UserModel.all().first().values())

{'id': 101, 'is_delete': False, 'created': datetime.datetime(2023, 8, 12, 18, 47, 52, 491896, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), 'modified': datetime.datetime(2023, 8, 12, 18, 47, 52, 491920, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), 'account': '0', 'password': '0', 'gender': <Gender.man: 1>, 'integral': Decimal('146.00000')}


In [23]:
# filter()
# 查询符合条件的数据
await UserModel.filter(id = 101)

[<UserModel: 101>]

In [28]:
# exclude()
# 查询排除符合条件的数据
print(await UserModel.all().count())
print(await UserModel.exclude(gender=1).count())

50
25


### filter 查询参数
* not

In [32]:
from tortoise.expressions import Q

# Q 或者查询
print(await UserModel.filter(Q(id=120)|Q(id=121)))
print(await UserModel.filter(Q(id=120),Q(id=121)))
print(await UserModel.filter(Q(id=120),~Q(id=121)))

[<UserModel: 120>, <UserModel: 121>]
[]
[<UserModel: 120>]
